In [2]:
import stim
from stim import PauliString, Tableau
from typing import List
from ucc_ft.checker import ft_check_ideal_qasm

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


This notebook applies the checker the state preparation circuit in "Measurement-free fault-tolerant logical zero-state encoding of the distance-three nine-qubit surface code in a one-dimensional qubit array" by Goto et. al [arxiv:2303.17211](http://arxiv.org/abs/2303.17211). As we will see, this paper has a slightly different definition of fault tolerance than this paper, and will be a useful example to consider any future extensions of the fault-checker tool.

The key image in the paper is:

![state preparation](fig-encoder.png)

It shows the state preparation circuit in figure (c), alongside the stabilizers at each step of the circuit execution. Note that gates within a step commute with each other.


To use `ucc-ft`,  we start by defining the stabilizers and logical operators used in the paper. Rather than using the arbitrary $d$ surface code class in `ucc_ft`, we just define the code explicitly for $d=3$ from the image above:

In [3]:
class RotatedSurfaceCodeFromPaper:
    """
    Just explicit version of the rotated surface code from the paper
    """

    def __init__(self):
        self.d = 3

    @property
    def num_qubits(self) -> int:
        """Return the number of qubits in the rotated surface code of distance d."""
        return self.d * self.d

    def stabilizers(self) -> List[PauliString]:
        return [
            PauliString("Z5*Z6"),
            PauliString("Z0*Z1*Z4*Z5"),
            PauliString("Z3*Z4*Z7*Z8"),
            PauliString("Z2*Z3"),
            PauliString("X0*X1"),
            PauliString("X1*X2*X3*X4"),
            PauliString("X4*X5*X6*X7"),
            PauliString("X7*X8"),
        ]

    def logical_x(self) -> PauliString:
        """Return the logical X operator for the rotated surface code of distance d."""
        return PauliString("X0*X5*X6")

    def logical_z(self) -> PauliString:
        """Return the logical Z operator for the rotated surface code of distance d."""
        return PauliString("Z0*Z1*Z2")

    def logical_prep_stabilizer(self) -> PauliString:
        """The prepared state is |0>_L, the +1 eigenstate of the logical Z operator."""
        return self.logical_z()

    def physical_z_stabilizers(self) -> List[PauliString]:
        """Return the physical Z operator for the rotated surface code of distance d."""
        return [PauliString(f"Z{idx}") for idx in range(self.num_qubits)]

In [4]:
sc = RotatedSurfaceCodeFromPaper()
sc.stabilizers()

[stim.PauliString("+_____ZZ"),
 stim.PauliString("+ZZ__ZZ"),
 stim.PauliString("+___ZZ__ZZ"),
 stim.PauliString("+__ZZ"),
 stim.PauliString("+XX"),
 stim.PauliString("+_XXXX"),
 stim.PauliString("+____XXXX"),
 stim.PauliString("+_______XX")]

Now, let's take a a look at the preparation circuit, defined in QASM. 

In [5]:
qasm = """
OPENQASM 3.0;
include "stdgates.inc";

qubit[9] data;

// Subroutine to do the state preperation circuit in Fig1c
def prepare_state() {

    // Step 0 in Fig1c
    for int i in [0:8] {
        reset data[i];
    }

    for int i in [1:2:8] {
        h data[i];
    }

    // Step 1 in Fig1c
    cx data[1], data[0];
    cx data[3], data[2];
    cx data[5], data[4];
    cx data[7], data[8];
    cx data[3], data[4];
    cx data[5], data[6];

    // Step 2 in Fig1c
    cx data[2], data[1];
    cx data[6], data[7];

}
"""

We can now run the checker to verify this gadget is fault-tolerant:

In [6]:
ft_check_ideal_qasm(sc, qasm, "prepare_state", "prepare", num_ancilla=0)

shape: (9, 18)
shape: (9, 18)
1
>>> 1 INIT target_qubit=1
2
>>> 2 INIT target_qubit=2
3
>>> 3 INIT target_qubit=3
4
>>> 4 INIT target_qubit=4
5
>>> 5 INIT target_qubit=5
6
>>> 6 INIT target_qubit=6
7
>>> 7 INIT target_qubit=7
8
>>> 8 INIT target_qubit=8
9
>>> 9 INIT target_qubit=9
10
>>> 11 H target_qubit=2
11
>>> 13 H target_qubit=4
12
>>> 15 H target_qubit=6
13
>>> 17 H target_qubit=8
14
>>> 21 CNOT target_qubit1=2, target_qubit2=1
15
>>> 25 CNOT target_qubit1=4, target_qubit2=3
16
>>> 29 CNOT target_qubit1=6, target_qubit2=5
17
>>> 33 CNOT target_qubit1=8, target_qubit2=9
18
>>> 37 CNOT target_qubit1=4, target_qubit2=5
19
>>> 41 CNOT target_qubit1=6, target_qubit2=7
20
>>> 45 CNOT target_qubit1=3, target_qubit2=2
21
>>> 49 CNOT target_qubit1=7, target_qubit2=8
>>> Fail!


[ Info: '`bitwuzla -rwl 1`' is used as smt solver for FT_condition case
[ Info: '`bitwuzla -rwl 1`' has solved the problem
[ Info: The assignment that generates the bug has been written to ./_temp_check_FT_condition_.output


False

The checker returned `False` indicating it found a set of errors that breaks the fault tolerance requirements! This seems like a contradiction with the results in the paper, so let's dig a little further.

For now, we can manually look at the output file of the SMT run to find the error assignments:

In [7]:
!grep b1 _temp_check_FT_condition_.output

  (define-fun nerrs_compact () (_ BitVec 1) #b1)
  (define-fun symb_Zerror_Q8_49 () (_ BitVec 1) #b1)
  (define-fun symb_Xerror_Q7_46 () (_ BitVec 1) #b1)
  (define-fun symb_Zerror_Q7_47 () (_ BitVec 1) #b1)
  (define-fun nerrs_all_compact () (_ BitVec 1) #b1)


These are all 1-bit vectors (the `( _ BitVec 1)`) that are assigned `1` (the `#b1`). All other variables in the solution have value 0.

The `nerrs_compact` and `nerrs_all_compact` confirms there was 1 error, which this distance 3 code should be able to handle. 

The remaining 3 variables are Z or X paulis, on qubit 7 or 8, and were the 46,47 and 49th injected Pauli error when running the circuit. The trace output above prints the current number of injected Paulis at each operation. So focusing on the last two operations 

```
>>> 45 CNOT target_qubit1=3, target_qubit2=2
21
>>> 49 CNOT target_qubit1=7, target_qubit2=8
```

shows the error was part of the last CNOT gate, and corresponds to an `X and Z` error on qubit 7 and a `Z` error on qubit 8. Note that even though this was expressed as 3 pauli injected errors, this correspond to one faulty operation (the CNOT), hence why it should be handled by the distance 3 code to be fault tolerance.


So the dilemma --this ssure looks like a weight-2 error, so how did the paper claim fault tolerance? Well, it turns out this weight-2 error is correctable by the code. 

First, looking back at the stabilizers, we see as a CSS code, each stabilizer is either all $X$ or all $Z$ operators and each is able to detect errors of the other variety (i.e. $X$ stabilizers detects phase errors $Z$). But these checks are independent, so they can be jointly detected and corrected. In other words, this means that any product of single $X$ and a single $Z$ error is correctable even if together that is a weight-2 error. The error here (ignoring the irrelevant global phase) is $X_7 Z_7 Z_8$. So the $X_7$ can be handled. 

What about $Z_7 Z_8$? This looks a two-qubit error that is not separable into single qubit errors. Well, turns out it is equivalent to a single-qubit error under this code. An error $E$ is equivalent to an error $E'$ if it's action on the stabilized state is the same, e.g. $E\psi\rangle = E'\psi\rangle$. If $E = P_i P_j$ on qubits $i$ and $j$, $E'=P_k$ could be a weight-1 error on qubit $k$ if we can find an operator $S$ of the stabilizer group such that $ES = E'$,

$$
P_iP_j |\psi\rangle = P_iP_j S |\psi\rangle = P_k |\psi\rangle
$$
From the stabilizers above, we see this is true for the last stabilizer $S = Z_7 Z_8 Z_9$. So $Z_7Z_8$ is equilavent to a $Z_9$ error on this code, and can be corrected alongside the $X_7$ error.

So why did the fault checker flag this? Let's go back to the definition of fault tolerance used by the checker. From Definition 6.1 in the fault checker [paper](http://arxiv.org/abs/2501.14380), a state preparation circuit gadget is fault tolerant if for any $s$-fault instantiation of the gadget such that $s \leq t$, the prepared state has at most $s$ errors. Here $t=1$, $s=1$ (just one faulty CNOT), but the prepared state has 2 errors. Even though the $t=1$ code _can_ correct the two errors in this case, that is not the definition used in the checker, which focuses purely on error propagation relative to $t$. This makes sense as a conservative approach, where each gadget independently conforms to the single threshold of the code.

Perhaps a future direction for the tool is to consider alternate definitions for fault tolerance to treat this case differently.
